In [0]:
!pip install pmdarima
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date
import pmdarima as pm
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA

In [0]:
df = pd.read_csv("/content/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv",delimiter= ',')
#df.tail()
#df.columns
#df['Seconds'] = df['Timestamp'].values.astype(float)

In [0]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit = 's')
df.set_index('Timestamp', inplace=True)
df = df.resample('D').mean()
df['Weighted_Price_box'], lmbda = stats.boxcox(df.Weighted_Price)

In [0]:
def difference(df, interval = 1):
  diff = list()
  for i in range(interval, len(df)):
    value = df[i] - df[i - interval]
    diff.append(value)
  return np.array(diff)

def inverse_difference(history, yhat, value = 1):
  return yhat + history[-value]

In [0]:
y = df.Weighted_Price.dropna()
diff = difference(y, 365)
model = ARIMA(diff, order = (1,1,2))
model_fit = model.fit(disp = 0)
print(model_fit.summary())

In [0]:
plt.rcParams.update({'figure.figsize':(12,5), 'figure.dpi':120})
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title = "Residuals", ax = ax[0])
residuals.plot(kind = "kde", title = "Density", ax = ax[1])
plt.show()

In [0]:
model_fit.plot_predict(dynamic = False)
plt.show()

In [0]:
#from datetime import date
#d1 = date(2019, 9, 1)
#d2 = (date.today() - d1).days +30

In [0]:
start_index = len(diff) - 30
end_index = start_index + 30
forecast = model_fit.predict(start = start_index, end = end_index)
history = [x for x in df.Weighted_Price]
day = 1
for yhat in forecast:
  inverted = inverse_difference(history, yhat, 365)
  print("Day %d : %f" % (day, inverted))
  history.append(inverted)
  day += 1
#for i in history[-30:]:
 # print(round(i, 2))